In [1]:
import os
import pandas as pd
import math
import plotly
import plotly.express as px

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import preprocessing

import umap
from umap import UMAP

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [2]:
# Read data
df = pd.read_parquet(
    "http://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/llm/summarization/llm_summarization.parquet"
)
baseline_df = df[:300]
recent_df = df[300:]
baseline_df = baseline_df.reset_index(drop=True)  
recent_df = recent_df.reset_index(drop=True)
baseline_df.head()

,prediction_timestamp,article,summary,reference_summary
0,2023-04-07 15:31:59.300109000,This is the moment when an angry woman forced ...,An elderly man was forced to kneel in the stre...,Man on moped knocked woman's phone out of her ...
1,2023-04-07 15:51:38.553976173,A Florida university is investigating the deat...,Bethune-Cookman University is investigating wh...,Bethune-Cookman University student Damian Park...
2,2023-04-07 16:11:17.807843347,"It was once the edgiest place in town, attract...","The Variety Theater in Cleveland, once a buzzi...",Cleveland's Variety Theater was a renowned roc...
3,2023-04-07 16:30:57.061710521,Unite baron Len McCluskey has boasted that tra...,"Len McCluskey, Unite's General Secretary, has ...",Unite boss Len McCluskey told members that Uni...
4,2023-04-07 16:50:36.315577695,Amazon is trialling a scheme that could spell...,Amazon is launching a pilot scheme in Munich i...,Pilot begins in May for Prime customers who d...


In [3]:
from rouge import Rouge
ROUGE = Rouge()
rouge_dict = ROUGE.get_scores(baseline_df["summary"], baseline_df["reference_summary"])

In [4]:
bdf = baseline_df.copy()
bdf["RougeDxnry"] = pd.Series(rouge_dict)
bdf['Rouge1'] = bdf.apply(lambda row: row.RougeDxnry['rouge-1']['f'], axis = 1)
bdf['Rouge2'] = bdf.apply(lambda row: row.RougeDxnry['rouge-2']['f'], axis = 1)
bdf['RougeL'] = bdf.apply(lambda row: row.RougeDxnry['rouge-l']['f'], axis = 1)
bdf.head()

,prediction_timestamp,article,summary,reference_summary,RougeDxnry,Rouge1,Rouge2,RougeL
0,2023-04-07 15:31:59.300109000,This is the moment when an angry woman forced ...,An elderly man was forced to kneel in the stre...,Man on moped knocked woman's phone out of her ...,"{'rouge-1': {'r': 0.4883720930232558, 'p': 0.3...",0.420000,0.113821,0.380000
1,2023-04-07 15:51:38.553976173,A Florida university is investigating the deat...,Bethune-Cookman University is investigating wh...,Bethune-Cookman University student Damian Park...,"{'rouge-1': {'r': 0.42391304347826086, 'p': 0....",0.503226,0.225806,0.477419
2,2023-04-07 16:11:17.807843347,"It was once the edgiest place in town, attract...","The Variety Theater in Cleveland, once a buzzi...",Cleveland's Variety Theater was a renowned roc...,"{'rouge-1': {'r': 0.4126984126984127, 'p': 0.4...",0.433333,0.078431,0.366667
3,2023-04-07 16:30:57.061710521,Unite baron Len McCluskey has boasted that tra...,"Len McCluskey, Unite's General Secretary, has ...",Unite boss Len McCluskey told members that Uni...,"{'rouge-1': {'r': 0.23255813953488372, 'p': 0....",0.219780,0.040000,0.175824
4,2023-04-07 16:50:36.315577695,Amazon is trialling a scheme that could spell...,Amazon is launching a pilot scheme in Munich i...,Pilot begins in May for Prime customers who d...,"{'rouge-1': {'r': 0.40425531914893614, 'p': 0....",0.339286,0.108844,0.285714


In [5]:
print (bdf.max())

prediction_timestamp                        2023-04-11 17:28:36.206393964
article                 With rigorous grooming regimes and Instagram f...
summary                 Zermatt, a town in Switzerland, has banned sel...
reference_summary       Yannick Bolasie raced past Lee Cattermole to c...
Rouge1                                                              0.625
Rouge2                                                           0.435644
RougeL                                                              0.625
dtype: object


In [5]:
def precision_range(metric_val):
    if metric_val < 0.1:
        return "Under 0.1"
    elif (metric_val >= 0.1) and (metric_val < 0.19):
        return "0.10-.19"
    elif (metric_val >= 0.2) and (metric_val < 0.29):
        return "0.20-.29"
    elif (metric_val >= 0.29) and (metric_val < 0.39):
        return "0.30-.39"
    elif (metric_val >= 0.39) and (metric_val < 0.49):
        return "0.40-.49"
    elif (metric_val >= 0.49) and (metric_val < 0.59):
        return "0.50-.59"
    elif (metric_val >= 0.59) and (metric_val < 0.69):
        return "0.60-.69"
    elif (metric_val >= 0.69) and (metric_val < 0.79):
        return "0.70-.79"
    elif (metric_val >= 0.79) and (metric_val < 0.89):
        return "0.80-.89"
    elif (metric_val >= 0.89) and (metric_val <= 1.0):
        return "0.9-1.0"
    
bdf['Rouge1Str'] = bdf.apply(lambda x: precision_range(x['Rouge1']), axis=1)
bdf['Rouge1Range'] = bdf.Rouge1Str.astype('category')

bdf['Rouge2Str'] = bdf.apply(lambda x: precision_range(x['Rouge2']), axis=1)
bdf['Rouge2Range'] = bdf.Rouge2Str.astype('category')

bdf['RougeLStr'] = bdf.apply(lambda x: precision_range(x['RougeL']), axis=1)
bdf['RougeLRange'] = bdf.RougeLStr.astype('category')

bdf = bdf.dropna()

bdf.head()

df_cluster = pd.DataFrame()

In [6]:
bdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 281 entries, 0 to 299
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   prediction_timestamp  281 non-null    datetime64[ns]
 1   article               281 non-null    object        
 2   summary               281 non-null    object        
 3   reference_summary     281 non-null    object        
 4   RougeDxnry            281 non-null    object        
 5   Rouge1                281 non-null    float64       
 6   Rouge2                281 non-null    float64       
 7   RougeL                281 non-null    float64       
 8   Rouge1Str             281 non-null    object        
 9   Rouge1Range           281 non-null    category      
 10  Rouge2Str             281 non-null    object        
 11  Rouge2Range           281 non-null    category      
 12  RougeLStr             281 non-null    object        
 13  RougeLRange         

***
# Clustering


In [9]:
def return_clustered_rep(rouge_id, rouge_df, df_cluster):
    vectorizer = CountVectorizer(min_df = 5, stop_words = 'english')
    word_doc_matrix = vectorizer.fit_transform(rouge_df.summary)
    word_doc_matrix

    umap_2d = UMAP(random_state=0)
    umap_2d.fit(word_doc_matrix)

    projections = umap_2d.transform(word_doc_matrix)
    
    df_cluster = pd.DataFrame(data = projections, columns = ['UmapComp1', 'UmapComp2'])
    
    if rouge_id == "ROUGE-1":
        df_cluster['RougeStr'] = rouge_df['Rouge1Str']
        df_cluster['Rouge Score'] = rouge_df['Rouge1']
    elif rouge_id == "ROUGE-2":
        df_cluster['RougeStr'] = rouge_df['Rouge2Str']
        df_cluster['Rouge Score'] = rouge_df['Rouge2']
    else :
        df_cluster['RougeStr'] = rouge_df['RougeLStr']
        df_cluster['Rouge Score'] = rouge_df['RougeL']
        

    df_cluster = df_cluster.dropna()
    
    df_cluster['IDStr'] = "DataPoint_" + df_cluster['UmapComp1'].index.astype(str)
    
    plot_df = df_cluster[['UmapComp1', 'UmapComp2', 'Rouge Score']]
    
    std_scaler = StandardScaler()
    cluster = std_scaler.fit_transform(plot_df.to_numpy())
    hover = df_cluster['IDStr']
    km = KMeans(random_state = 42, n_init = 10, max_iter=100)
    km.fit(cluster)
    df_cluster['label'] = km.labels_
    df_cluster = df_cluster.round(decimals = 5)
    
    df_cluster.to_csv('/home/gatha/cluster.csv')
    rouge_fig = px.scatter_3d(df_cluster, x='UmapComp1', y='UmapComp2', z='Rouge Score', color = df_cluster['Rouge Score'], height = 800, width = 1000, hover_name = hover)

    rouge_fig.update_layout(dragmode='select',
                            activeselection=dict(fillcolor='yellow'))

    return df_cluster, rouge_fig

def get_filtered_fig(value, ):
    global bdf
    global df_cluster
    
    max_score = df_cluster['Rouge Score'].max()
    min_score = df_cluster['Rouge Score'].min()
    gap = (max_score - min_score) / 4
    print ("Max: ", max_score, "Min: ", min_score, "gap: ", gap)
    
    plot_df = df_cluster[['UmapComp1', 'UmapComp2', 'Rouge Score', 'IDStr']]
    
    if value == "HIGH":
        up_bound = max_score - gap
        plot_df = df_cluster[df_cluster['Rouge Score'] > up_bound]
        marker_shade = '#FBE426'
    elif value == "MODERATE":
        up_bound = max_score - gap
        low_bound = min_score + (2 * gap)
        plot_df = df_cluster[(df_cluster['Rouge Score'] <= up_bound) & (df_cluster['Rouge Score'] > low_bound)]
        marker_shade = '#F58518'
    elif value == "LOW":
        up_bound = max_score - (2 * gap)
        low_bound = min_score + gap
        plot_df = df_cluster[(df_cluster['Rouge Score'] <= up_bound) & (df_cluster['Rouge Score'] > low_bound)]
        marker_shade = '#1616A7'
    elif value == "SEVERE":
        low_bound = min_score + gap
        plot_df = df_cluster[df_cluster['Rouge Score'] <= low_bound]
        marker_shade = '#782AB6'
       
    hover = plot_df['IDStr']

    filtered_fig = px.scatter_3d(plot_df,
                                 x='UmapComp1',
                                 y='UmapComp2',
                                 z='Rouge Score',
                                 height = 800,
                                 width = 1000,
                                 hover_name = hover).update_traces(marker = dict(color = marker_shade))

    filtered_fig.update_layout(dragmode='select',
                    activeselection=dict(fillcolor='yellow'))

    return filtered_fig
    

In [10]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_table as dt
from pprint import pprint
from dash import exceptions

# Build App
#df = px.data.tips()
rouge_nms = ["ROUGE-1", "ROUGE-2", "ROUGE-L"]

app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([
        html.H1("Your Model's Summarization Peformance"),
        html.Div([
            # 1. Drop-down of Location names
            html.H2("Computed Rouge Scores"),
            html.Label([
                "Please Select One",
                dcc.Dropdown(
                    id = 'rouge_dropdown',
                    clearable = False,
                    value = rouge_nms[0],
                    options = [
                        {'label': c, 'value': c}
                        for c in rouge_nms
                    ]),
            ]),
        ]),
    ]),
    
    html.Div([
        html.Div(
        dcc.Graph(
                id="clustered_rouge", 
                responsive=True,
                style={
                    "width": "100%",
                    "height": "100%"
                }
            ),
            style={
                "width": "68%",
                "height": "800px",
                "display": "inline-block",
                "border": "1px #5c5c5c solid",
                "padding-top": "5px",
                "padding-left": "1px",
                "overflow": "hidden"
            }
        ),
        html.Div([
            html.H4("Performance Filters for Embeddings"),
            dcc.RadioItems(
                id='radio_items',
                options=[
                {'label': 'High', 'value': 'HIGH'},
                {'label': 'Moderate', 'value': 'MODERATE'},
                {'label': 'Bad', 'value': 'LOW'},
                {'label': 'Severe', 'value': 'SEVERE'}
                ],
                value='HIGH'
            ),
            html.Button('Submit', id='submit_button'),
            html.Button('Reset', id='reset_button'),
        ]),
    ]),
    
    html.Div(
    html.Div(
        id="table_container",
    ),
    style={
            "width": "68%",
            "height": "800px",
            "display": "inline-block",
            "border": "1px #5c5c5c solid",
            "padding-bottom": "100px",
            "padding-left": "1px",
            "overflow": "hidden"
        },
    ),
])

@app.callback(
    Output('clustered_rouge', 'figure'),
    [Input("rouge_dropdown", "value")]
)
def update_rouge_clusters(locn):
    global bdf
    global df_cluster

    active_rouge = locn
    case_fig = None
    
    if locn == "ROUGE-1":
        rouge_df = bdf[["Rouge1", "article", "summary", "Rouge1Str"]]
        df_cluster, case_fig = return_clustered_rep("ROUGE-1", rouge_df, df_cluster)
        return case_fig
    elif locn == "ROUGE-2":
        rouge_df = bdf[["Rouge2", "article", "summary", "Rouge2Str"]]
        df_cluster, case_fig = return_clustered_rep("ROUGE-2", rouge_df, df_cluster)
        return case_fig
    else:
        rouge_df = bdf[["RougeL", "article", "summary", "RougeLStr"]]
        df_cluster, case_fig = return_clustered_rep("ROUGE-L", rouge_df, df_cluster)
        return case_fig

    return case_fig

# Filter data-points for chosen performance category
@app.callback(
    Output('clustered_rouge', 'figure', allow_duplicate=True),
    [Input('submit_button', 'n_clicks')],
    [State('radio_items', 'value')],
    prevent_initial_call = True)
def update_output(n_clicks, value):
    print('The input value was "{}" and the button has been clicked {} times'.format(
            value,
            n_clicks
        )
    )
    global bdf
    global df_cluster
    
    filtered_fig = get_filtered_fig(value)
    return filtered_fig


# Reset the graph to show all embeddings
@app.callback(
    Output('clustered_rouge', 'figure', allow_duplicate=True),
    [Input('reset_button', 'n_clicks')],
    prevent_initial_call = True
    )
def reset_output(n_clicks):
    global bdf
    global df_cluster

    case_fig = None
    
    print ("Reset rouge", active_rouge)
    
    if active_rouge == "ROUGE-1":
        rouge_df = bdf[["Rouge1", "article", "summary", "Rouge1Str"]]
        df_cluster, case_fig = return_clustered_rep("ROUGE-1", rouge_df, df_cluster)
        return case_fig
    elif active_rouge == "ROUGE-2":
        rouge_df = bdf[["Rouge2", "article", "summary", "Rouge2Str"]]
        df_cluster, case_fig = return_clustered_rep("ROUGE-2", rouge_df, df_cluster)
        return case_fig
    else:
        active_rouge = bdf[["RougeL", "article", "summary", "RougeLStr"]]
        df_cluster, case_fig = return_clustered_rep("ROUGE-L", rouge_df, df_cluster)
        return case_fig

    return case_fig

@app.callback(
    Output("table_container", "children"),
    Input("clustered_rouge", "clickData"),
)
def fig_click(clickData):
    if not clickData:
        raise exceptions.PreventUpdate

    global bdf
    global df_cluster
    
    sel_datapt = clickData["points"][0]["hovertext"]
    sel_row = df_cluster.loc[df_cluster["IDStr"].str.contains(sel_datapt)]
    row_df = bdf.loc[sel_row.index]
    row_df = row_df[['summary', 'reference_summary']]
    tcols = [{"name": i, "id": i} for i in row_df.columns]
    return html.Div([
        dt.DataTable(
            id='summaries_table',
            columns = tcols,
            data = row_df.to_dict("rows"),
            style_cell={'textAlign': 'left', 'padding': '5px', 'whiteSpace': 'normal', 'height': 'auto'},
            style_header = {
                'fontWeight': 'bold'
            }
        )
    ])

if __name__ == '__main__':
    active_rouge = "ROUGE-1"
    app.run_server(mode='external')

Dash is running on http://127.0.0.1:8050/

Dash app running on http://127.0.0.1:8050/
